In [2]:
!pip install einops
!pip install gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.4 MB/s eta 0:00:00


In [3]:
# Importing the libraries
import torch
from torch import nn, einsum
import torch.nn.functional as F

from einops import rearrange, repeat
from einops.layers.torch import Rearrange

import gdown

In [25]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
import torch.nn as nn
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torchvision.transforms import ToTensor
# from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from tqdm import tqdm, trange
from torchvision import transforms

In [28]:
# helpers

def pair(t):
    return t if isinstance(t, tuple) else (t, t)

# classes

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head *  heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.attend = nn.Softmax(dim = -1)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        b, n, _, h = *x.shape, self.heads
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = h), qkv)

        dots = einsum('b h i d, b h j d -> b h i j', q, k) * self.scale

        attn = self.attend(dots)

        out = einsum('b h i j, b h j d -> b h i d', attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
            ]))
    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return x

class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels = 3, dim_head = 64, dropout = 0., emb_dropout = 0.):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)

        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
            nn.Linear(patch_dim, dim),
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            # nn.Linear(dim, num_classes) # Output (batch_size, num_classes)
            nn.Linear(dim, 128) # 128 desired embedding size (?)
        )

    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '() n d -> b n d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x)
        return self.mlp_head(x)

In [29]:
# Loss function
class RankingLoss:

    def __init__(self):
        pass

    def _label2similarity(sekf, label1, label2):
        '''
        compute similarity matrix of label1 and label2
        :param label1: torch.Tensor, [m]
        :param label2: torch.Tensor, [n]
        :return: torch.Tensor, [m, n], {0, 1}
        '''
        m, n = len(label1), len(label2)
        l1 = label1.view(m, 1).expand([m, n])
        l2 = label2.view(n, 1).expand([n, m]).t()
        similarity = l1 == l2
        return similarity

    def _batch_hard(self, mat_distance, mat_similarity, more_similar):

        if more_similar is 'smaller':
            sorted_mat_distance, _ = torch.sort(mat_distance + (-9999999.) * (1 - mat_similarity), dim=1, descending=True)
            hard_p = sorted_mat_distance[:, 0]
            sorted_mat_distance, _ = torch.sort(mat_distance + (9999999.) * (mat_similarity), dim=1, descending=False)
            hard_n = sorted_mat_distance[:, 0]
            return hard_p, hard_n

        elif more_similar is 'larger':
            sorted_mat_distance, _ = torch.sort(mat_distance + (9999999.) * (1 - mat_similarity), dim=1, descending=False)
            hard_p = sorted_mat_distance[:, 0]
            sorted_mat_distance, _ = torch.sort(mat_distance + (-9999999.) * (mat_similarity), dim=1, descending=True)
            hard_n = sorted_mat_distance[:, 0]
            return hard_p, hard_n


class TripletLoss(RankingLoss):
    '''
    Compute Triplet loss augmented with Batch Hard
    Details can be seen in 'In defense of the Triplet Loss for Person Re-Identification'
    Args:
        margin(float or 'soft'): if float, use nn.MarginRankingLoss, if 'soft',
    '''

    def __init__(self, margin, metric, reduce=True):
        '''
        :param margin: float or 'soft', for MarginRankingLoss with margin and soft margin
        :param bh: batch hard
        :param metric: l2 distance or cosine distance
        '''

        assert isinstance(margin, float) or margin=='soft', \
            'margin must be type float or value \'soft\', but got {}'.format(margin)
        if isinstance(margin, float):
            self.margin_loss = nn.MarginRankingLoss(margin=margin, reduce=reduce)
        elif margin == 'soft':
            self.margin_loss = nn.SoftMarginLoss(reduce=reduce)
        self.metric = metric

    def __call__(self, emb, label):
        '''
        :param emb: torch.Tensor, [m, dim]
        :param label: torch.Tensor, [b]
        '''

        if self.metric == 'cosine':
            mat_dist = cosine_dist(emb, emb)
            mat_sim = self._label2similarity(label, label)
            hard_p, _ = self._batch_hard(mat_dist, mat_sim.float(), more_similar='larger')

            mat_dist = cosine_dist(emb, emb)
            mat_sim = self._label2similarity(label, label)
            _, hard_n = self._batch_hard(mat_dist, mat_sim.float(), more_similar='larger')

            margin_label = -torch.ones_like(hard_p)

            # print(len(emb)) # batch-size
            # print(len(label))
            # print('Mat_dist : ', mat_dist)


        elif self.metric == 'euclidean':
            mat_dist = euclidean_dist(emb, emb)
            mat_sim = self._label2similarity(label, label)
            hard_p, _ = self._batch_hard(mat_dist, mat_sim.float(), more_similar='smaller')

            mat_dist = euclidean_dist(emb, emb)
            mat_sim = self._label2similarity(label, label)
            _, hard_n = self._batch_hard(mat_dist, mat_sim.float(), more_similar='smaller')

            margin_label = torch.ones_like(hard_p)

        if self.margin_loss.__class__.__name__ == 'MarginRankingLoss':
            return self.margin_loss(hard_n, hard_p, margin_label)
        elif self.margin_loss.__class__.__name__ == 'SoftMarginLoss':
            return self.margin_loss(hard_n-hard_p, margin_label)


def cosine_dist(x, y):
    '''
    compute cosine distance between two matrix x and y
    with size (n1, d) and (n2, d) and type torch.tensor
    return a matrix (n1, n2)
    '''

    x = F.normalize(x, p=2, dim=1)
    y = F.normalize(y, p=2, dim=1)
    return torch.matmul(x, y.transpose(0, 1))


def euclidean_dist(x, y):
    """
    compute euclidean distance between two matrix x and y
    with size (n1, d) and (n2, d) and type torch.tensor
    return a matrix (n1, n2)
    """
    m, n = x.size(0), y.size(0)
    xx = torch.pow(x, 2).sum(1, keepdim=True).expand(m, n)
    yy = torch.pow(y, 2).sum(1, keepdim=True).expand(n, m).t()
    dist = xx + yy
    dist.addmm_(1, -2, x, y.t())
    dist = dist.clamp(min=1e-12).sqrt()  # for numerical stability
    return dist

<>:22: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:29: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:22: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:29: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-29-cea9566bfff4>:22: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if more_similar is 'smaller':
<ipython-input-29-cea9566bfff4>:29: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif more_similar is 'larger':


In [ ]:
# file_id = "0B8-rUzbwVRk0c054eEozWG9COHM"
# url = f"https://drive.google.com/uc?id={file_id}"

# output = "Market-1501.zip"  # Specify the desired name for the output file

# gdown.download(url, output, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=0B8-rUzbwVRk0c054eEozWG9COHM
To: /content/Market-1501.zip
100%|██████████| 153M/153M [00:02<00:00, 58.4MB/s]


'Market-1501.zip'

In [ ]:
# import zipfile

# with zipfile.ZipFile(output, 'r') as zip_ref:
#     zip_ref.extractall("/content/drive/MyDrive/ViT/Data-Market/")  # Specify the desired folder for extraction


In [26]:
# # # Prepare-market.py
# import os
# from shutil import copyfile

# # You only need to change this line to your dataset download path
# download_path = '/content/drive/MyDrive/ViT/Data-Market/Market-1501-v15.09.15'

# if not os.path.isdir(download_path):
#     print('please change the download_path')

# save_path = download_path + '/pytorch'
# if not os.path.isdir(save_path):
#     os.mkdir(save_path)
# #-----------------------------------------
# #query
# query_path = download_path + '/query'
# query_save_path = download_path + '/pytorch/query'
# if not os.path.isdir(query_save_path):
#     os.mkdir(query_save_path)

# for root, dirs, files in os.walk(query_path, topdown=True):
#     for name in files:
#         if not name[-3:]=='jpg':
#             continue
#         ID  = name.split('_')
#         src_path = query_path + '/' + name
#         dst_path = query_save_path + '/' + ID[0]
#         if not os.path.isdir(dst_path):
#             os.mkdir(dst_path)
#         copyfile(src_path, dst_path + '/' + name)

# #-----------------------------------------
# #multi-query
# query_path = download_path + '/gt_bbox'
# # for dukemtmc-reid, we do not need multi-query
# if os.path.isdir(query_path):
#     query_save_path = download_path + '/pytorch/multi-query'
#     if not os.path.isdir(query_save_path):
#         os.mkdir(query_save_path)

#     for root, dirs, files in os.walk(query_path, topdown=True):
#         for name in files:
#             if not name[-3:]=='jpg':
#                 continue
#             ID  = name.split('_')
#             src_path = query_path + '/' + name
#             dst_path = query_save_path + '/' + ID[0]
#             if not os.path.isdir(dst_path):
#                 os.mkdir(dst_path)
#             copyfile(src_path, dst_path + '/' + name)

# #-----------------------------------------
# #gallery
# gallery_path = download_path + '/bounding_box_test'
# gallery_save_path = download_path + '/pytorch/gallery'
# if not os.path.isdir(gallery_save_path):
#     os.mkdir(gallery_save_path)

# for root, dirs, files in os.walk(gallery_path, topdown=True):
#     for name in files:
#         if not name[-3:]=='jpg':
#             continue
#         ID  = name.split('_')
#         src_path = gallery_path + '/' + name
#         dst_path = gallery_save_path + '/' + ID[0]
#         if not os.path.isdir(dst_path):
#             os.mkdir(dst_path)
#         copyfile(src_path, dst_path + '/' + name)

# #---------------------------------------
# #train_all
# train_path = download_path + '/bounding_box_train'
# train_save_path = download_path + '/pytorch/train_all'
# if not os.path.isdir(train_save_path):
#     os.mkdir(train_save_path)

# for root, dirs, files in os.walk(train_path, topdown=True):
#     for name in files:
#         if not name[-3:]=='jpg':
#             continue
#         ID  = name.split('_')
#         src_path = train_path + '/' + name
#         dst_path = train_save_path + '/' + ID[0]
#         if not os.path.isdir(dst_path):
#             os.mkdir(dst_path)
#         copyfile(src_path, dst_path + '/' + name)


# #---------------------------------------
# #train_val
# train_path = download_path + '/bounding_box_train'
# train_save_path = download_path + '/pytorch/train'
# val_save_path = download_path + '/pytorch/val'
# if not os.path.isdir(train_save_path):
#     os.mkdir(train_save_path)
#     os.mkdir(val_save_path)

# for root, dirs, files in os.walk(train_path, topdown=True):
#     for name in files:
#         if not name[-3:]=='jpg':
#             continue
#         ID  = name.split('_')
#         src_path = train_path + '/' + name
#         dst_path = train_save_path + '/' + ID[0]
#         if not os.path.isdir(dst_path):
#             os.mkdir(dst_path)
#             dst_path = val_save_path + '/' + ID[0]  #first image is used as val image
#             os.mkdir(dst_path)
#         copyfile(src_path, dst_path + '/' + name)

In [30]:
import os
from torch.utils.data import Dataset
from skimage import io

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None, train=True):
        self.root_dir = root_dir
        self.transform = transform
        self.train = train
        self.image_paths, self.labels = self._load_dataset()

    def _load_dataset(self):
        image_paths = []
        labels = []

        # Select train or val folder based on self.train
        data_folder = 'train' if self.train else 'val'

        for person_id in os.listdir(os.path.join(self.root_dir, data_folder)):
            person_folder = os.path.join(self.root_dir, data_folder, person_id)

            if os.path.isdir(person_folder):
                for filename in os.listdir(person_folder):
                    if filename.endswith('.jpg'):
                        image_paths.append(os.path.join(person_folder, filename))
                        labels.append(int(person_id))

        return image_paths, labels

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        img_path = self.image_paths[index]
        image = io.imread(img_path)
        label = self.labels[index]

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
dataset_root = '/content/drive/MyDrive/ViT/Data-Market/Market-1501-v15.09.15/'

# Create the dataset and dataloader
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 128)),
    transforms.ToTensor(),
])

custom_dataset = CustomDataset(root_dir=dataset_root, transform=transform)
dataloader = DataLoader(custom_dataset, batch_size=32, shuffle=True)


In [ ]:
def main():
    # Set your device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    dataset_root = '/content/drive/MyDrive/ViT/Data-Market/Market-1501-v15.09.15/pytorch/'

    # Create the dataset and dataloader
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 128)),
        transforms.ToTensor(),
    ])

    train_dataset = CustomDataset(root_dir=dataset_root, transform=transform, train=True)
    val_dataset = CustomDataset(root_dir=dataset_root, transform=transform, train=False)

    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # Define your vision transformer model
    model = ViT(
        image_size=(256, 128),
        patch_size=(16, 16),
        num_classes=1501,
        dim=256,
        depth=12,
        heads=8,
        mlp_dim=512,
        dropout=0.1,
        emb_dropout=0.1
    )

    model.to(device)

    # Define your optimizer and loss function
    optimizer = Adam(model.parameters(), lr=1e-4)
    loss_fn = TripletLoss(margin='soft', metric='cosine')

    # Training loop
    num_epochs = 10  # Set the number of epochs

    for epoch in range(num_epochs):
      # Training
      model.train()
      total_loss = 0.0
      num_batches = len(train_dataloader)
      for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1} in training", leave=False):
          images, labels = batch
          optimizer.zero_grad()
          outputs = model(images)
          # print('outputs')
          # print(outputs)
          # print(len(outputs))
          loss = loss_fn(outputs, labels)
          print(loss)
          loss.backward()
          optimizer.step()
          print("labels: ", len(labels))
          print("Images: ", len(images))
          total_loss += loss.item()
      average_loss = total_loss / num_batches

      print(f'Training Epoch [{epoch+1}/{num_epochs}], Average Loss: {average_loss}')
      print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

      # Validation
      model.eval()
      total_val_loss = 0.0
      num_val_batches = len(val_dataloader)

      with torch.no_grad():
          for val_batch in tqdm(val_dataloader, desc=f"Epoch {epoch + 1} in validation", leave=False):
              val_images, val_labels = val_batch
              val_outputs = model(val_images)
              val_loss = loss_fn(val_outputs, val_labels)
              total_val_loss += val_loss.item()

      average_val_loss = total_val_loss / num_val_batches
      print(f'Validation Epoch [{epoch+1}/{num_epochs}], Average Loss: {average_val_loss}')

    # Save the trained model
    torch.save(model.state_dict(), '/content/drive/MyDrive/ViT/Model/model_v1.pth')


if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
Epoch 1 in training:   0%|          | 0/405 [00:00<?, ?it/s]

tensor(0.6411, grad_fn=<SoftMarginLossBackward0>)


Epoch 1 in training:   0%|          | 1/405 [00:06<40:54,  6.07s/it]

labels:  32
Images:  32
tensor(0.5817, grad_fn=<SoftMarginLossBackward0>)


Epoch 1 in training:   0%|          | 2/405 [00:13<46:10,  6.88s/it]

...

labels:  32
Images:  32
tensor(0.4084, grad_fn=<SoftMarginLossBackward0>)


Epoch 8 in training:   5%|▍         | 20/405 [02:01<37:51,  5.90s/it]

labels:  32
Images:  32
tensor(0.4236, grad_fn=<SoftMarginLossBackward0>)


Epoch 8 in training:   5%|▌         | 21/405 [02:08<39:58,  6.25s/it]

labels:  32
Images:  32


In [ ]:
def main():
    # Loading data
    transform = ToTensor()

    train_set = MNIST(root='./../datasets', train=True, download=True, transform=transform)
    test_set = MNIST(root='./../datasets', train=False, download=True, transform=transform)

    train_loader = DataLoader(train_set, shuffle=True, batch_size=128)
    test_loader = DataLoader(test_set, shuffle=False, batch_size=128)

    # Defining model and training options
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device: ", device, f"({torch.cuda.get_device_name(device)})" if torch.cuda.is_available() else "")
    model = ViT(image_size=(28, 28), patch_size=(7, 7), num_classes=10, dim=8, depth=2, heads=2, mlp_dim=8, pool='cls', channels=1, dim_head=64, dropout=0.1, emb_dropout=0.1).to(device)
    N_EPOCHS = 5
    LR = 0.005

    optimizer = Adam(model.parameters(), lr=LR)
    criterion = CrossEntropyLoss()

    for epoch in trange(N_EPOCHS, desc="Training"):
        train_loss = 0.0
        model.train()

        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1} in training", leave=False):
            x, y = batch
            x, y = x.to(device), y.to(device)
            y_hat = model(x)
            loss = criterion(y_hat, y)

            train_loss += loss.detach().cpu().item() / len(train_loader)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{N_EPOCHS} loss: {train_loss:.2f}")

    # Test loop
    with torch.no_grad():
        correct, total = 0, 0
        test_loss = 0.0
        model.eval()

        for batch in tqdm(test_loader, desc="Testing"):
            x, y = batch
            x, y = x.to(device), y.to(device)
            y_hat = model(x)
            loss = criterion(y_hat, y)
            test_loss += loss.detach().cpu().item() / len(test_loader)

            correct += torch.sum(torch.argmax(y_hat, dim=1) == y).detach().cpu().item()
            total += len(x)

        print(f"Test loss: {test_loss:.2f}")
        print(f"Test accuracy: {correct / total * 100:.2f}%")

In [ ]:
if __name__ == '__main__':
  main()

100%|██████████| 9912422/9912422 [00:00<00:00, 109221216.12it/s]


Extracting ./../datasets/MNIST/raw/train-images-idx3-ubyte.gz to ./../datasets/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 26564845.14it/s]

Extracting ./../datasets/MNIST/raw/train-labels-idx1-ubyte.gz to ./../datasets/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 22089853.70it/s]


...

Epoch 4/5 loss: 0.66



Training: 100%|██████████| 5/5 [02:13<00:00, 26.66s/it]


Epoch 5/5 loss: 0.63


Testing: 100%|██████████| 79/79 [00:02<00:00, 36.33it/s]


Test loss: 0.39
Test accuracy: 87.62%
